In [ ]:
import os

os.makedirs("input", exist_ok=True)
os.makedirs("intermediate", exist_ok=True)
os.makedirs("output", exist_ok=True)

MEDS_ETL_OUTPUT = "output"
GRAPH_OUTPUT_PATH = f"{MEDS_ETL_OUTPUT}/rdf"

In [2]:
from utils.pre_MEDS import generate_meds_preprocessed
from utils.synthetic_generator import generate_synthetic_dataset

df_input = generate_synthetic_dataset(10000, output_csv="input/syn_data.csv")
df_intermediate = generate_meds_preprocessed(df_input, output_path="intermediate")

In [3]:
from MEDS_transforms.runner import main
import shutil

shutil.rmtree(MEDS_ETL_OUTPUT)

main([
    "pkg://MEDS_extract.configs._extract.yaml",
    "--overrides",
    "input_dir=intermediate",
    "output_dir=output",
    "event_conversion_config_fp=MESSY.yaml",
    "dataset.name=Neurovasc",
    "dataset.version=1.0",
])

0

In [4]:
from meds2rdf import MedsRDFConverter

shacl_graph = "https://raw.githubusercontent.com/TeamHeKA/meds-ontology/refs/heads/main/shacl/meds-shapes.ttl"

# Initialize the converter with the path to your MEDS dataset directory
converter = MedsRDFConverter("output")

# Convert the dataset into an RDF graph
graph = converter.convert(
    include_splits=True,
)

In [6]:
os.makedirs(GRAPH_OUTPUT_PATH, exist_ok=True)
# Serialize the graph to different formats
graph.serialize(destination=f"{GRAPH_OUTPUT_PATH}/output_dataset.ttl", format="turtle")
#graph.serialize(destination=f"{OUTPUT_DIR}/output_dataset.xml", format="xml")
#graph.serialize(destination=f"{OUTPUT_DIR}/output_dataset.nt", format="nt")

print("Conversion complete! RDF files saved.")

Conversion complete! RDF files saved.


In [7]:
from utils.metrics import collect_all_metrics_from_output, save_stats_json

metrics = collect_all_metrics_from_output(
    output_path=MEDS_ETL_OUTPUT, 
    csv_path="input/syn_data.csv", 
    graph_path=f"{GRAPH_OUTPUT_PATH}/output_dataset.ttl"
)

save_stats_json(metrics, "metrics/all_metrics.json")